In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import random
import os
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
%run ./Combine_csv.ipynb


In [3]:
import datetime

now = datetime.datetime.now()


Today_time = now.strftime("%H:%M")

Today_date = now.strftime("%Y-%m-%d")

In [4]:
prefix_dir ='Combined CSV/'
suffix_dir = 'combined_lifestyle-'+Today_date+'.csv'
today_csv=os.path.join(prefix_dir+suffix_dir)

In [5]:
lifestyle = pd.read_csv(today_csv)

In [6]:
lifestyle.head()

,Source,Heading,Category,Date,Time,URL
0,India Today,Kiara Advani trolled for wearing fake designe...,fashion,2019-06-21,15:02:00,https://www.indiatoday.in/lifestyle/fashion/st...
1,India Today,Priyanka Chopra stuns in metallic sheer top a...,fashion,2019-06-21,15:02:00,https://www.indiatoday.in/lifestyle/fashion/st...
2,India Today,Priyanka Chopra paints NYC tangerine in plung...,fashion,2019-06-21,15:02:00,https://www.indiatoday.in/lifestyle/fashion/st...
3,India Today,Priyanka Chopra is all about summer romance i...,fashion,2019-06-21,15:02:00,https://www.indiatoday.in/lifestyle/fashion/st...
4,India Today,Sunny Leone is all summer shine and glory in ...,fashion,2019-06-21,15:02:00,https://www.indiatoday.in/lifestyle/fashion/st...


In [7]:
lifestyle.shape

(983, 6)

In [8]:
df = lifestyle
Heading = df['Heading']

In [9]:
df['Category'].value_counts()

fashion                     179
beauty, health & fitness    135
travel                      109
parenting                    98
relationships                70
motoring                     63
books                        56
food                         40
life & style                 39
others                       36
celebrity                    36
home & style                 32
homes and gardens            31
luxury                       30
life                         29
Name: Category, dtype: int64

### Tokenizing and Stemming

In [10]:
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re

In [11]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\drago\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\drago\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
stopset = set(stopwords.words('english'))

In [13]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [14]:
df['Heading'] = df['Heading'].apply(remove_apostrophe)
df['Heading'] = df['Heading'].apply(remove_punctuation)
df['Heading'] = df['Heading'].apply(convert_numbers)

In [15]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [16]:
def tokenize_and_stem(text):
   
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []

    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):

    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
   
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [17]:
Heading_stemmed = []
Heading_tokenized = []
for i in Heading:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'Heading', tokenize/stem
    Heading_stemmed.extend(allwords_stemmed) #extend the 'Heading_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    Heading_tokenized.extend(allwords_tokenized)

In [18]:
words_frame = pd.DataFrame({'words': Heading_tokenized}, index = Heading_stemmed)

In [19]:
words_frame.head(10)

,words
kiara,kiara
advani,advani
troll,trolled
for,for
wear,wearing
fake,fake
design,designer
dress,dress
copi,copy
toh,toh


### Spliting and applying algorithms


In [20]:
def get_best_n(tfidf_matrix,dist):
    
    n_clusters = list (range (14,20))
    min=999
    for n in n_clusters:
        km_ss = KMeans(n_clusters=n)
        clusters_ss = km_ss.fit_predict(tfidf_matrix)
        score = silhouette_score(dist,clusters_ss)
        if min>score:
            min=score
            n_score=n
    return n_score

In [21]:
df_dict={}
for cat in df['Category'].unique():
    temp = df[df['Category']==cat].reset_index().drop(['index'],axis=1)
    Heading = temp['Heading']
    vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,4))
    tfidf_matrix = vectorizer.fit_transform(Heading)
    
    dist = cosine_similarity(tfidf_matrix)

    n = get_best_n(tfidf_matrix,dist)
    km = KMeans(n_clusters=n)
    km.fit(tfidf_matrix)
    
    clusters = km.labels_.tolist()
    temp['Cluster'] = clusters
 
    df_sorted=temp.sort_values(by='Cluster').reset_index()
    df_sorted.drop(['index'],axis=1,inplace=True)
    
    grp = df_sorted.sort_values('Cluster').groupby(['Cluster'],as_index=False)
    
    
    no_of_clusters=len(grp)

    cluster_similarity_value =[]
    

    vectorizer = TfidfVectorizer()
    for i in range(no_of_clusters):
        group = grp.get_group(i)

        cluster_heading=group['Heading']
        cluster_matrix = vectorizer.fit_transform(cluster_heading)
        cluster_dist = cosine_similarity(cluster_matrix)
        cluster_elements_count = pd.DataFrame.count(group)
        x=[]
        for i in cluster_dist:

            if((cluster_elements_count[0]-1)==0):
                y=1
            else:
                y=float("{0:.2f}".format(((i.sum())/(cluster_elements_count[0]))))
            x.append(y)
            cluster_similarity_value.append(y)
   
    
    
    df_sorted['cluster_similarity_value']=cluster_similarity_value;  col=df_sorted.columns
    
    grp = df_sorted.sort_values('Cluster').groupby(['Cluster'],as_index=False)
    temp_more =[]
    temp_less  =[]
    for i in range(no_of_clusters):
        cluster = grp.get_group(i)
        cluster_mean = cluster['cluster_similarity_value'].mean()
        cluster_std = cluster['cluster_similarity_value'].std()
        comp_fact = cluster_mean + cluster_std/4
        for i in range(len(cluster)):
            if (cluster.iloc[i]['cluster_similarity_value']<comp_fact):
                temp_less.append(cluster.iloc[i])
            else:
                temp_more.append(cluster.iloc[i])
    df_more_similar=pd.DataFrame(temp_more,columns=col)
    df_less_similar=pd.DataFrame(temp_less,columns=col)
    
    
    Result = df_more_similar.sort_values('Cluster').groupby(['Cluster'],as_index=False).apply(lambda x: x.sample(1)) 
    Result = Result.reset_index().drop(['level_0','level_1'],axis=1)
    Result = Result.append(df_less_similar)
    Result = Result.sort_values(by='Cluster')
    Result = Result.reset_index().drop(['index'],axis=1)
    
    
    df_dict[cat] = Result
    

In [22]:
df_dict['fashion']

,Source,Heading,Category,Date,Time,URL,Cluster,cluster_similarity_value
0,India Today,Priyanka Chopra Jonas on her evolving style I...,fashion,2019-06-21,15:02:00,https://www.indiatoday.in/lifestyle/fashion/st...,0,0.15
1,India Today,Priyanka Chopra teaches how to slay sexy athl...,fashion,2019-06-21,15:02:00,https://www.indiatoday.in/lifestyle/fashion/st...,0,0.09
2,India Today,Priyanka Chopra stuns in metallic sheer top a...,fashion,2019-06-21,15:02:00,https://www.indiatoday.in/lifestyle/fashion/st...,0,0.12
3,Savvy,Priyanka Chopra Nick Jonas Wedding Reception ...,fashion,2018-12-22,15:02,http://www.savvy.co.in/article/fashion/priyank...,0,0.13
4,Time of India,Celeb style Tamannaahs summer date night look...,fashion,2019-06-18,14:04:00,https://timesofindia.indiatimes.com/life-style...,0,0.08
5,The Luxe Cafe,Summer Frocks For The Summer Child In You,fashion,2019-06-21,15:02,http://www.theluxecafe.com/stories/summer-froc...,0,0.12
6,Time of India,Buzz This is Nita Ambanis favourite colour,fashion,2019-06-20,13:40:00,https://timesofindia.indiatimes.com/life-style...,0,0.13
7,India Today,Priyanka Chopra paints NYC tangerine in plung...,fashion,2019-06-21,15:02:00,https://www.indiatoday.in/lifestyle/fashion/st...,0,0.13
8,The Hindu,Cannes 2019 red carpet Priyanka Chopra Deepik...,fashion,2019-05-17,20:05:00,https://www.thehindu.com/life-and-style/fashio...,0,0.08
9,Time of India,Celeb style Priyanka Chopra is wearing this o...,fashion,2019-06-17,16:00:00,https://timesofindia.indiatimes.com/life-style...,0,0.09


In [23]:
for typ,data in df_dict.items():
    
    outname =typ +'.csv'
    root = 'Categorized Output/'
    if not os.path.exists(root):
        os.mkdir(root)
    date_today= Today_date +'/'
    outdir=root+ date_today[:-1]
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname)
    data.to_csv(fullname,index=False,encoding='utf-8')